# Data Preparation and Persistence
---
In this notebook, csv data is prepared and stored in a database for persistence.

### Transactions_By_Dept Data
---
###### source: https://data.world/zpencer/transaction-itemset

In [2]:
import pandas as pd
from sqlalchemy import create_engine, BigInteger, Integer, String

# Pull data from csv to DataFrame
transactions_by_dept_path = 'data/transactions_by_dept.csv'
transactions_df = pd.read_csv(transactions_by_dept_path)
transactions_df.columns = ['TransactionId', 'Department','ItemId','SalesUnits']
transactions_df = transactions_df.sort_values(['TransactionId','ItemId'])

# Map TransactionId to smaller values
transaction_id_map = pd.DataFrame(transactions_df['TransactionId'].unique(), columns=['TransactionId'])
transaction_id_map['Id'] = transaction_id_map.index+1
transactions_df = transactions_df.merge(transaction_id_map, on='TransactionId') \
                                 .drop('TransactionId',axis=1) \
                                 .rename(columns={'Id': 'TransactionId'})

transactions_df['LineId'] = transactions_df.groupby(['TransactionId']).cumcount()+1
transactions_df = transactions_df.set_index(['TransactionId','LineId'])
transactions_df.head()

# Data is now ready to be written to database for persistence

Department  ItemId  SalesUnits
TransactionId LineId                                               
1             1       0634:VITAMINS & HLTH AIDS     102           1
              2               0879:PET SUPPLIES     158           2
              3                      0973:CANDY     175           2
              4                    0982:SPIRITS     176           1
              5                       0983:WINE     177           4

In [3]:
# Establish a connection
host = 'localhost'
database = 'data'
driver = 'SQL+Server+Native+Client+11.0' # Driver found in ODBC Data Source Administrator app
engine = create_engine("mssql+pyodbc://{0}/{1}?driver={2}".format(host,database,driver))

# Specify schema and table
schema = 'dbo'
table_name = 'TransactionsByDept'

# Define data types (not always necessary)
data_types = {'TransactionId': Integer,
              'LineId': Integer,
              'Department': String,
              'ItemId': Integer,
              'SalesUnits': Integer}

# Write dataframe to a table
transactions_df.to_sql(table_name,con=engine,schema=schema,
                       index=True, index_label=['TransactionId','LineId'],
                       if_exists='replace',dtype=data_types)

### Extended_Bakery Data
---
###### source: https://github.com/Stocco/Data-Mining-the-Extended-Bakery

In [25]:
import pandas as pd
from sqlalchemy import create_engine, BigInteger, Integer, String

extended_bakery_path = 'data/75000-out1.csv'
bakery_items_path = "data/EB-build-goods.sql"

########################################################
# Read receipts data from file into an array of strings
# Receipts data takes the following form:
#   ReceiptID, ItemID_1, ItemID_2, ..., ItemID_N
with open(extended_bakery_path,'r') as file:
    bakery_data = file.read().split('\n')

list_bakery_transactions = [line.split(", ") for line in bakery_data[0:-1]]
base_data = list()
for bakery_transaction in list_bakery_transactions:
    base_data += (list(map(lambda e: (bakery_transaction[0],e),bakery_transaction[1:])))
bakery_df = pd.DataFrame(base_data,columns=['TransactionId','ItemId'])
bakery_df['LineId'] = bakery_df.groupby(['TransactionId']).cumcount()+1
bakery_df = bakery_df.set_index(['TransactionId','LineId'])
bakery_df

###############################################
# Pull product data from sql file to DataFrame
#   "insert into goods values (0,'Chocolate','Cake',8.95,'Food');"
with open(bakery_items_path,'r') as file:
    lines = itemsFile.read().split('\n')
    
# For each line in lines less the last line,
#   Get data after the '(' and split by ','
#   Drop the last 2 items (price, category)
items = [line.split('(')[1].split(',')[0:-2] for line in lines[0:-1]]

# Map items list into a list of tuples with concatenated item description
# (ItemID, ItemName)
itemMap = [(item[0], "{0} {1}".format(item[1],item[2]).replace("'","")) for item in items]
item_df = pd.DataFrame(itemMap,columns=['ItemId', 'ItemName']).set_index('ItemId')


# Data is now ready to be written to database for persistence

In [ ]:
# Establish a connection
host = 'localhost'
database = 'data'
driver = 'SQL+Server+Native+Client+11.0' # Driver found in ODBC Data Source Administrator app
engine = create_engine("mssql+pyodbc://{0}/{1}?driver={2}".format(host,database,driver))

############################
# Specify schema and table
schema = 'dbo'
table_name = 'BakeryTransaction'

# Define data types (not always necessary)
data_types = {'TransactionId': Integer,
              'LineId': Integer,
              'ItemId': Integer}

# Write dataframe to a table
bakery_df.to_sql(table_name, con=engine, schema=schema,
                 index=True, index_label=['TransactionId','LineId'],
                 if_exists='replace', dtype=data_types)

###########################
# Specify schema and table
schema = 'dbo'
table_name = 'BakeryItem'

# Define data types (not always necessary)
data_types = {'ItemId': Integer,
              'ItemName': String}

# Write dataframe to a table
item_df.to_sql(table_name, con=engine, schema=schema,
               index=True, index_label=['ItemId'],
               if_exists='replace', dtype=data_types)

### Order_Products Data
---
###### source: https://www.kaggle.com/c/instacart-market-basket-analysis/data

In [1]:
import pandas as pd
from sqlalchemy import create_engine, BigInteger, Integer, String

order_products_train_path = 'data/order_products__train.csv'
order_products_prior_path = 'data/order_products__prior.csv'
products_path = 'data/products.csv'

##################################
# Pull data from csv to DataFrame
transactions_train_df = pd.read_csv(order_products_train_path)

transactions_train_df.columns = ['TransactionId','ItemId','AddToCardOrder','Reordered']
transactions_train_df = transactions_train_df.sort_values(['TransactionId','ItemId'])

transactions_train_df['LineId'] = transactions_train_df.groupby(['TransactionId']).cumcount()+1
transactions_train_df = transactions_train_df.set_index(['TransactionId','LineId'])
transactions_train_df.head()

##################################
# Pull data from csv to DataFrame
transactions_prior_df = pd.read_csv(order_products_prior_path)

transactions_prior_df.columns = ['TransactionId','ItemId','AddToCardOrder','Reordered']
transactions_prior_df = transactions_prior_df.sort_values(['TransactionId','ItemId'])

transactions_prior_df['LineId'] = transactions_prior_df.groupby(['TransactionId']).cumcount()+1
transactions_prior_df = transactions_prior_df.set_index(['TransactionId','LineId'])
transactions_prior_df.head()

####################################
# Union two transactions dataframes
transactions_df = pd.concat([transactions_train_df, transactions_prior_df])


##################################
# Pull data from csv to DataFrame
items_df = pd.read_csv(products_path)
items_df.columns = ['ItemId','ItemName','AisleId','DepartmentID']
items_df = items_df.set_index('ItemId')
items_df.head()

# Data is now ready to be written to database for persistence

,ItemName,AisleId,DepartmentID
ItemId,,,
1,Chocolate Sandwich Cookies,61,19
2,All-Seasons Salt,104,13
3,Robust Golden Unsweetened Oolong Tea,94,7
4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
5,Green Chile Anytime Sauce,5,13


In [ ]:
# Establish a connection
host = 'localhost'
database = 'data'
driver = 'SQL+Server+Native+Client+11.0' # Driver found in ODBC Data Source Administrator app
engine = create_engine("mssql+pyodbc://{0}/{1}?driver={2}".format(host,database,driver))

############################
# Specify schema and table
schema = 'dbo'
table_name = 'ProductTransaction'

# Define data types (not always necessary)
data_types = {'TransactionId': Integer,
              'LineId': Integer,
              'ItemId': Integer,
              'AddToCartOrder': Integer,
              'Reordered': Integer}

# Write dataframe to a table
transactions_df.to_sql(table_name, con=engine, schema=schema,
                       index=True, index_label=['TransactionId','LineId'],
                       if_exists='replace', dtype=data_types, chunksize=200000)

###########################
# Specify schema and table
schema = 'dbo'
table_name = 'Product'

# Define data types (not always necessary)
data_types = {'ItemId': Integer,
              'ItemName': String,
              'AisleId': Integer,
              'DepartmentId': Integer}

# Write dataframe to a table
items_df.to_sql(table_name, con=engine, schema=schema,
               index=True, index_label=['ItemId'],
               if_exists='replace', dtype=data_types, chunksize=200000)